In [194]:
import math
import random
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from itertools import permutations

In [195]:
#-------------------------Génération de la solution initiale en utilisant K-means------------------------
def kmeans_imp(X, clusters):
    # Number of clusters
    kmeans = KMeans(n_clusters=clusters)
    # Fitting the input data
    kmeans = kmeans.fit(X)
    # Getting the cluster labels
    labels = kmeans.predict(X)
    # Centroid values
    centroids = kmeans.cluster_centers_
    unique_elements, counts_elements = np.unique(labels, return_counts=True)
    return labels,counts_elements

In [196]:
#-------------------------Calcul de l'énergie de la solution------------------------
def f(x,y,effectif,general_center,I):
    #Calculate between-class scatter
    clf = NearestCentroid()
    clf.fit(x, y)
    centroids=clf.centroids_
    var=centroids-general_center
    var=var**2
    effectif=effectif/np.shape(X)[0]
    var=var*effectif[:, np.newaxis]
    B=np.sum(var)
    #Calculate total scatter
    return B/I

In [197]:
#-------------------------Calcul de l'intertue total------------------------
def inertie_total(x,general_center):
    #Calculate total scatter
    total=x-general_center
    total=total**2
    I= sum(np.mean(total,axis=0))
    return I

In [198]:
#-------------------------Calcul du centre de gravité du dataset------------------------
def gravity_center(x):
        return np.mean(x,axis=0)

In [199]:
#-------------------------Perturbation de la solution courante------------------------
def perturb(p, effectif,l):
    #effectif: tableau ou liste des effectifs par labels, de meme longueur que l (et contient le meme ordre des labels)   
    effectifs=effectif.copy()
    curr = p.copy()
    m= np.shape(X)[0]
    perturbed_state = p.copy() #state upon which the action is done
    i = random.randint(0,m-1) #choose a random object 
    while (True):
        index = random.randint(0,len(l)-1)
        perturbed_state[i] = l[index]
        if(perturbed_state[i] != p[i]):
            # check whether the disturbed state is different from current_state*
            effectifs[p[i]] = effectifs[p[i]] - 1
            effectifs[ l[index]] = effectifs[ l[index]] + 1
            break 
    return perturbed_state,effectifs  # return state upon which the action is done.

In [200]:
#*********************************Recuit Simulé ***************************
def Recuit_Simulé(max_iter, temperature_init, alpha, temperature_fin, état_init,effectif, étiquettes_inutilisés):
    
    #Intialement on considère la température initiale comme température courante
    t = temperature_init
    #Et l'état initial comme état courant
    état_courant = état_init.copy()
    #Calcule de centre de gravité 
    general_center=gravity_center(X)
    I=inertie_total(X,general_center)
    f_courant=f(X,état_courant,effectif,general_center,I)
    print("L'état courant:", état_courant)
    print("L'énergie de l'état courant:", f_courant)
    print("Effectif:", effectif)
    #Le critère d’arrêt est exprimé sous la forme de la température finale
    while(t >= temperature_fin):
        #Pour chaque température on va exécuter l'algorithme max_iter fois
        for i in range(1, max_iter):
            #On sélectionne une solution voisine en utilisant la stratégie de sélection "perturb" afin de mettre à l'état 
            #courant
            état_suivant, effectif_suivant = perturb(état_courant,effectif,étiquettes_inutilisés)
             #On teste si la nouvelle solution est meilleure
             #Si ce n'est pas le cas on va l'accepter avec une certaine probabilité 
            f_suivant= f(X,état_suivant, effectif_suivant,general_center,I)
            énergie_delta = f_suivant - f_courant
            if ((énergie_delta >0) or (math.exp( énergie_delta / t) <= random.uniform(0,1))):
                état_courant = état_suivant
                f_courant= f_suivant
                effectif=effectif_suivant
        #On met à jour la température en appliquant la fonction géométrique 
        t = alpha * t
    return état_courant,f_courant,effectif


In [201]:
#*********************************Génération aléatoire de la solustion initiale ***************************
def generation_aleatoire(m,k):
    solution=[]
    effectif=[0]*k
    for i in range(m):
        u=random.uniform(0,1)
        solution.append(math.floor(u*k))
        effectif[math.floor(u*k)]=effectif[math.floor(u*k)]+1
    return np.array(solution), np.array(effectif)

In [202]:
#*********************************Algorithme général avec solution initial aléatoire ***************************
def Recuit_Simulé_init_aleatoire(X,max_iter, temperature_init, alpha, temperature_fin,nb_clusters):
    m=np.shape(X)[0]
    etat_init,effectif=generation_aleatoire(m,nb_clusters)
    l=[j for j in (range(nb_clusters))]
    solution,f_solution,effectif=Recuit_Simulé(max_iter, temperature_init, alpha, temperature_fin, etat_init,effectif, l)
    return solution,f_solution,effectif
   

In [203]:
#*********************************Algorithme général avec solution initial avec Kmeans ***************************
def Recuit_Simulé_init_Kmeans(X,max_iter, temperature_init, alpha, temperature_fin,nb_clusters):
    m=np.shape(X)[0]
    etat_init,effectif=kmeans_imp(X,nb_clusters)
    l=[j for j in (range(nb_clusters))]
    solution,f_solution,effectif=Recuit_Simulé(max_iter, temperature_init, alpha, temperature_fin, etat_init,effectif,l)
    return solution,f_solution,effectif
   

In [225]:
#*********************************Cacul de taux d'érreur ***************************
def taux_erreurs(Y_true,Y_result):
    return 1-accuracy_score(Y_true, Y_result, normalize=True)

In [226]:
#*********************************lier les classes trouvé aux classes réels ***************************
def match_classes(true_y,solution_y,true_classes,solution_classes):
    max_accurcy=0
    matched_solution=[]
    match_classes=solution_classes.copy()
    m=len(true_classes)
    per = list(permutations(true_classes))
    for i in range(len(per)):
        possibility=per[i]
        match_solution=[] 
        for it in range(len(true_y)):
            index=solution_y[it]
            match_solution.append(possibility[index])
        curr_acc=accuracy_score(true_y, match_solution, normalize=True)
        if(curr_acc>max_accurcy):
            max_accurcy=curr_acc
            matched_solution=match_solution
    return matched_solution

In [227]:
#*********************************Trouvé les objets mal classé et leur nombre ***************************
def objet_mal_classe(true_y,solution_y):
    objet=[]
    for i in range(len(true_y)):
        if(true_y[i]!=solution_y[i]):
            objet.append(i)
    return objet

In [241]:
#***************générer le fichier iris csv pour faire ACP ***************************
def iris_to_csv(dataset,solution_y):
    dataset["variety"]=solution_y
    dataset.to_csv(path)

In [243]:
#***************générer le fichier heart csv pour faire ACP ***************************
def heart_to_csv(dataset,solution_y,path):
    dataset["target"]=solution_y
    dataset.to_csv(path)

In [246]:
#Test aleatoire iris
iris = pd.read_csv("iris.csv")
X = iris.iloc[:, :-1].values;
y = iris.iloc[:,-1].values
solution,f_solution,effectif=Recuit_Simulé_init_aleatoire(X,6284, 2296,0.587 , 78,3)
print("L'état Final:",solution)
print("L'énergie de l'état Final:", f_solution)
print("Effectif:", effectif)
solution=match_classes(y,solution,["Setosa","Virginica","Versicolor"],[0,1,2])
print(taux_erreurs(y,solution))
len(objet_mal_classe(y,solution))
iris_to_csv(iris,solution,"iris_cluster_aleatoire.csv")

L'état courant: [1 2 0 1 0 2 0 2 2 0 1 1 2 2 2 0 2 1 1 2 1 0 1 2 2 0 2 2 1 0 1 2 0 0 1 1 1
 1 0 0 0 1 2 2 1 1 1 1 0 0 0 0 1 1 2 2 0 0 1 1 1 1 1 0 2 2 1 2 2 0 1 2 0 1
 0 0 2 1 2 2 0 0 0 2 2 0 2 0 2 2 2 0 1 1 1 1 1 1 0 2 2 1 2 1 1 2 2 0 1 1 1
 2 1 0 0 0 1 1 1 0 0 2 1 2 1 2 2 0 2 2 1 2 1 1 1 2 0 2 1 0 0 0 0 1 1 2 1 0
 1 2]
L'énergie de l'état courant: 0.0030174159903024584
Effectif: [44 57 49]
L'état Final: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 1 1 2 1 1 1 1
 1 1 2 2 1 1 1 1 2 1 2 1 2 1 1 2 2 1 1 1 1 1 2 1 1 1 1 2 1 1 1 2 1 1 1 2 1
 1 2]
L'énergie de l'état Final: 0.8842752513446485
Effectif: [50 38 62]
0.10666666666666669


In [249]:
#Test kmeans iris 
solution,f_solution,effectif=Recuit_Simulé_init_Kmeans(X,6284, 2296,0.587 , 78,3)
print("L'état Final:",solution)
print("L'énergie de l'état Final:", f_solution)
print("Effectif:", effectif)
solution=match_classes(y,solution,["Setosa","Virginica","Versicolor"],[0,1,2])
print(taux_erreurs(y,solution))
print(len(objet_mal_classe(y,solution)))
iris_to_csv(iris,solution,"iris_cluster_Kmeans.csv")

L'état courant: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 0 2 0 0 0 0
 0 0 2 2 0 0 0 0 2 0 2 0 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 2 0 0 0 2 0 0 0 2 0
 0 2]
L'énergie de l'état courant: 0.8842752513446485
Effectif: [38 50 62]
L'état Final: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 0 2 0 0 0 0
 0 0 2 2 0 0 0 0 2 0 2 0 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 2 0 0 0 2 0 0 0 2 0
 0 2]
L'énergie de l'état Final: 0.8842752513446485
Effectif: [38 50 62]
0.10666666666666669
16


In [253]:
#Test aleatoire heart
heart = pd.read_csv("heart.csv")
X = heart.iloc[:, :-1].values;
y = heart.iloc[:,-1].values
solution,f_solution,effectif=Recuit_Simulé_init_aleatoire(X,5920, 6282,0.9 , 72,2)
print("L'état Final:",solution)
print("L'énergie de l'état Final:", f_solution)
print("Effectif:", effectif)
solution=match_classes(y,solution,[0,1],[0,1])
print(taux_erreurs(y,solution))
print(len(objet_mal_classe(y,solution)))
heart_to_csv(heart,solution,"heart_cluster_aleatoire.csv")

L'état courant: [0 0 1 1 0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 0 1 0 0 0 0
 1 1 1 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1
 1 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0
 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1
 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1
 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1 1 0 0
 1 1 1 1 1 0 0 0 1 1 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 1 1
 1 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 1 1
 1 1 0 0 0 0 0]
L'énergie de l'état courant: 0.002786410181580255
Effectif: [159 144]
L'état Final: [1 1 1 1 0 1 0 0 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0
 1 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0
 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 0 0 1 0 0 1 1 1 0 

In [254]:
#Test aleatoire heart
heart = pd.read_csv("heart.csv")
X = heart.iloc[:, :-1].values;
y = heart.iloc[:,-1].values
solution,f_solution,effectif=Recuit_Simulé_init_Kmeans(X,5920, 6282,0.9 , 72,2)
print("L'état Final:",solution)
print("L'énergie de l'état Final:", f_solution)
print("Effectif:", effectif)
solution=match_classes(y,solution,[0,1],[0,1])
print(taux_erreurs(y,solution))
print(len(objet_mal_classe(y,solution)))
heart_to_csv(heart,solution,"heart_cluster_kmneans.csv")

L'état courant: [0 0 0 0 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1
 0 1 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0
 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0
 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 0
 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0
 0 0 0 1 0 0 0]
L'énergie de l'état courant: 0.44434376398985864
Effectif: [193 110]
L'état Final: [0 0 0 0 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1
 0 1 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1